# 서울시 구별 주유소

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('서울_주유소(220418).csv')
df.head()

,상호,주소,가격,셀프,상표,구
0,방죽주유소,서울 강남구 밤고개로 215 (율현동),1977,Y,GS칼텍스,강남구
1,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),1979,Y,SK에너지,강남구
2,현대오일뱅크 도곡셀프주유소,서울 강남구 남부순환로 2718 (도곡2동),1983,Y,현대오일뱅크,강남구
3,오일프러스 셀프,서울 강남구 남부순환로 2651 (도곡동),1983,Y,SK에너지,강남구
4,현대오일뱅크(주)직영 산성셀프주유소,서울 강남구 헌릉로 730,1985,Y,현대오일뱅크,강남구


In [3]:
gu_mm = df.groupby('구')['가격'].agg(['mean','min','max'])
gu_mm.head()

,mean,min,max
구,,,
강남구,2131.848485,1977,2548
강동구,2072.928571,1969,2298
강북구,1960.461538,1934,2039
강서구,1984.156250,1902,2348
관악구,2004.214286,1949,2087


In [4]:
_, mi, ma = gu_mm.loc['강남구']
mi, ma = int(mi), int(ma)
mi, ma

(1977, 2548)

In [5]:
df['mm'] = np.zeros(df.shape[0], int)
df.head(3)

,상호,주소,가격,셀프,상표,구,mm
0,방죽주유소,서울 강남구 밤고개로 215 (율현동),1977,Y,GS칼텍스,강남구,0
1,(주)보성 세곡주유소,서울 강남구 헌릉로 731 (세곡동),1979,Y,SK에너지,강남구,0
2,현대오일뱅크 도곡셀프주유소,서울 강남구 남부순환로 2718 (도곡2동),1983,Y,현대오일뱅크,강남구,0


In [6]:
for gu in gu_mm.index:
    _, mi, ma = gu_mm.loc[gu]
    mi, ma = int(mi), int(ma)
    for i in df.index:
        if df.가격[i] == mi and df.구[i] == gu:
            df.mm[i] = 1
        if df.가격[i] == ma and df.구[i] == gu:
            df.mm[i] = 2

In [7]:
# 구별 최소, 최대값이 아닌 주유소는 배제
df = df[df.mm != 0]
df.head()

,상호,주소,가격,셀프,상표,구,mm
0,방죽주유소,서울 강남구 밤고개로 215 (율현동),1977,Y,GS칼텍스,강남구,1
32,(주)만정에너지,서울 강남구 봉은사로 433 (삼성동),2548,N,GS칼텍스,강남구,2
33,삼미북부셀프주유소,서울 도봉구 도봉로 437 (쌍문동),1945,Y,SK에너지,도봉구,1
34,(주)자연에너지 햇살주유소,서울 도봉구 방학로 142 (방학동),1945,Y,현대오일뱅크,도봉구,1
35,(주)헨지스에너지 구도일주유소 도봉,서울 도봉구 도봉로 706 (방학동),1945,Y,S-OIL,도봉구,1


In [8]:
df.shape

(70, 7)

- 카카오 로컬 활용하여 좌표 구하기

In [9]:
import requests
from urllib.parse import quote

In [10]:
with open('../04.지도시각화/kakaoapikey.txt') as file:
    kakao_key = file.read()

In [11]:
local_url = 'https://dapi.kakao.com/v2/local/search/address.json'
header = {'Authorization': f'KakaoAK {kakao_key}'}

In [12]:
lng_list, lat_list = [], []
for i in df.index:
    url = f'{local_url}?query={quote(df.주소[i])}'
    result = requests.get(url, headers=header).json()
    lng_list.append(float(result['documents'][0]['x']))
    lat_list.append(float(result['documents'][0]['y']))

In [13]:
df['위도'] = lat_list
df['경도'] = lng_list
df.head(3)

,상호,주소,가격,셀프,상표,구,mm,위도,경도
0,방죽주유소,서울 강남구 밤고개로 215 (율현동),1977,Y,GS칼텍스,강남구,1,37.474998,127.106869
32,(주)만정에너지,서울 강남구 봉은사로 433 (삼성동),2548,N,GS칼텍스,강남구,2,37.511551,127.047115
33,삼미북부셀프주유소,서울 도봉구 도봉로 437 (쌍문동),1945,Y,SK에너지,도봉구,1,37.644972,127.032159


- 지도 위에 주유소 표시하기

In [14]:
import folium, json

In [15]:
filename = '../04.지도시각화/skorea_municipalities_geo_simple.json'
geo_str = json.load(open(filename, encoding='utf-8'))

In [16]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=12)
for i in df.index:
    folium.Marker(
        location=[df.위도[i], df.경도[i]],
        popup=folium.Popup(df.주소[i], max_width=200),
        tooltip=f'{df.상호[i]}<br>휘발유: {df.가격[i]:,d}원',
        icon=folium.Icon(color='green' if df.mm[i] == 1 else 'red',
                         icon='glyphicon glyphicon-thumbs-up' if df.mm[i] == 1 \
                                  else 'glyphicon glyphicon-thumbs-down')
    ).add_to(map)
map

- Choropleth 위에 표시하기

In [17]:
map = folium.Map(location=[37.5502, 126.982], zoom_start=12, tiles='Stamen Toner')
folium.Choropleth(
    geo_data=geo_str,
    data=gu_mm['mean'],
    columns=[gu_mm.index, gu_mm['mean']],
    fill_color='PuRd',
    key_on='feature.id'
).add_to(map)
for i in df.index:
    folium.Marker(
        location=[df.위도[i], df.경도[i]],
        popup=folium.Popup(df.주소[i], max_width=200),
        tooltip=f'{df.상호[i]}<br>휘발유: {df.가격[i]:,d}원',
        icon=folium.Icon(color='green' if df.mm[i] == 1 else 'red',
                         icon='glyphicon glyphicon-thumbs-up' if df.mm[i] == 1 \
                                  else 'glyphicon glyphicon-thumbs-down')
    ).add_to(map)   
map